## Homework: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

### Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

In [1]:
from sentence_transformers import SentenceTransformer

/Users/patrickcmd/Projects/personal/llm-zoomcamp/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
vec = embedding_model.encode(user_question)

In [5]:
len(vec)

768

In [7]:
vec[0]

0.07822263

### Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [9]:
len(documents)

948

In [10]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

In [11]:
def filter_documents_by_course(documents, course_name):
    """
    Filters the documents based on the given course name.

    Parameters:
    documents (list): A list of dictionaries where each dictionary represents a document.
    course_name (str): The name of the course to filter the documents by.

    Returns:
    list: A list of dictionaries that belong to the specified course.
    """
    return [doc for doc in documents if doc.get('course') == course_name]

In [12]:
ml_documents = filter_documents_by_course(documents, "machine-learning-zoomcamp")

In [13]:
len(ml_documents)

375

In [14]:
ml_documents[:2]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?'

### Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses.

In [15]:
from tqdm.auto import tqdm

In [56]:
qa_text_embeddings = []

for doc in tqdm(ml_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f"{question} {text}"

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qa_text)
    qa_text_embeddings.append(embedding_model.encode(qa_text))

100%|██████████| 375/375 [01:53<00:00,  3.29it/s]


In [17]:
len(qa_text_embeddings)

375

In [19]:
import numpy as np

In [24]:
X = np.array(qa_text_embeddings)

X.shape

(375, 768)

### Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065


In [27]:
scores = X.dot(vec)

scores.max()

0.65065753

### Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [28]:
class VectorSearchEngine:
    """
    A vector search engine that retrieves the most relevant documents based on vector similarity.

    Attributes:
    -----------
    documents : list
        A list of documents where each document is represented as a dictionary.
    embeddings : numpy.ndarray
        A 2D numpy array where each row is the vector embedding of a document.

    Methods:
    --------
    __init__(documents, embeddings)
        Initializes the search engine with documents and their corresponding embeddings.
    search(v_query, num_results=10)
        Searches for and returns the most relevant documents based on the query vector.
    """

    def __init__(self, documents, embeddings):
        """
        Initializes the VectorSearchEngine with a list of documents and their embeddings.

        Parameters:
        -----------
        documents : list
            A list of documents where each document is represented as a dictionary.
        embeddings : numpy.ndarray
            A 2D numpy array where each row is the vector embedding of a document.
        """
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        """
        Searches for and returns the most relevant documents based on the query vector.

        Parameters:
        -----------
        v_query : numpy.ndarray
            A vector representing the query.
        num_results : int, optional
            The number of top relevant documents to return (default is 10).

        Returns:
        --------
        list
            A list of the most relevant documents.
        """
        scores = self.embeddings.dot(v_query)  # Calculate the similarity scores
        idx = np.argsort(-scores)[:num_results]  # Get indices of the top `num_results` scores
        return [self.documents[i] for i in idx]  # Return the corresponding documents


In [29]:
search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine.search(vec, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33


In [30]:
import pandas as pd

In [31]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']


In [32]:
df_ground_truth.head()

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


In [34]:
df_ground_truth.dropna(inplace=True)

In [36]:
df_ground_truth.shape

(1830, 3)

In [37]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [39]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

#### Evaluation metrics for the VectorSearch Enginee using HitRate

In [38]:
# hit rate (recall)
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [52]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        user_question = q["question"]
        vec = embedding_model.encode(user_question)
        results = search_function(vec, num_results=5)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    
    print(results[-1])
    print(relevance_total[0])
    print(relevance_total[-1])

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [53]:
search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)

evaluate(ground_truth, search_engine.search)

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [01:21<00:00, 22.36it/s]

{'text': 'Install Nvidia drivers: https://www.nvidia.com/download/index.aspx.\nWindows:\nInstall Anaconda prompt https://www.anaconda.com/\nTwo options:\nInstall package ‘tensorflow-gpu’ in Anaconda\nInstall the Tensorflow way https://www.tensorflow.org/install/pip#windows-native\nWSL/Linux:\nWSL: Use the Windows Nvida drivers, do not touch that.\nTwo options:\nInstall the Tensorflow way https://www.tensorflow.org/install/pip#linux_1\nMake sure to follow step 4 to install CUDA by environment\nAlso run:\necho ‘export XLA_FLAGS=--xla_gpu_cuda_data_dir=$CONDA_PREFIX/lib/> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh\nInstall CUDA toolkit 11.x.x https://developer.nvidia.com/cuda-toolkit-archive\nInstall https://developer.nvidia.com/rdp/cudnn-download\nNow you should be able to do training/inference with GPU in Tensorflow\n(Learning in public links Links to social media posts where you share your progress with others (LinkedIn, Twitter, etc). Use #mlzoomcamp tag. The scores for this part 

{'hit_rate': 0.9398907103825137}

### Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [54]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "ml-course-questions-vector"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-course-questions-vector'})

In [57]:
for doc in tqdm(ml_documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:04<00:00, 82.37it/s] 


In [61]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'question_text_vector': array([ 8.80591646e-02,  1.55936433e-02,  7.92557746e-02,  2.52757091e-02,
         7.55764470e-02, -3.90596166e-02, -4.13813293e-02,  2.52917074e-02,
         2.43241899e-02,  3.62590281e-03, -7.28286617e-03, -3.28751355e-02,
         6.12956062e-02, -5.71100414e-02,  1.16774524e-02, -1.79441012e-02,
         4.49205674e-02, -5.41606285e-02, -1.92252628e-03,  1.48329502e-02,
         7.91352708e-03, -3.43126953e-0

In [60]:
user_question = "I just discovered the course. Can I still join it?"
v_q = embedding_model.encode(user_question)

In [62]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [63]:
search_results = elastic_search_knn("question_text_vector", v_q, "machine-learning-zoomcamp")

In [64]:
search_results

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

### Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

In [65]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [66]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [67]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 1830/1830 [01:22<00:00, 22.06it/s]


{'hit_rate': 0.9398907103825137}